In [1]:
import sys, os
import requests
import json
import numpy as np
import time
import pandas as pd
import datetime as dt
from config import api_key_inCites
from config import api_key_WOS
import clarivate.wos_journals.client
from clarivate.wos_journals.client.api import journals_api
from clarivate.wos_journals.client.model.journal_list import JournalList
from pprint import pprint

In [2]:
## Print files in current dirctory
for i in os.listdir('./'):
    print(i)

.git
.gitignore
.ipynb_checkpoints
CMMS_Faculty_Publications_Scopus_WOS.ipynb
CMMS_ILL_Requests_Turnaways.ipynb
config.py
data
LICENSE
output
README.md
__pycache__


In [3]:
## Use this to SQL query in Elements Reporting Database for faculty publication data
## https://github.com/kglibrarian/symplecticelementssql/blob/master/Data%20Source%20Publication%20IDs%20Report%20by%20Group%20of%20Authors.md

## Upload the resulting .csv from Elements

elements_data_path = "data/2024_05-16_FSM_Pubs_2019_to_2023.csv"
##elements_data_path = "data/2023_01-12_FSM_Pubs_2018_to_2022.csv"

## Read the CSV file and store into Pandas DataFrame 
elements_data_df = pd.read_csv(elements_data_path, encoding = "ISO-8859-1")
## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
elements_data_df.columns =  elements_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
elements_data_df.head()

C:\Users\keg827\AppData\Local\Temp\ipykernel_24616\1519900468.py:10: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  elements_data_df = pd.read_csv(elements_data_path, encoding = "ISO-8859-1")
C:\Users\keg827\AppData\Local\Temp\ipykernel_24616\1519900468.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  elements_data_df.columns =  elements_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","").str.replace("ï»¿","")
C:\Users\keg827\AppData\Local\Temp\ipykernel_24616\1519900468.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  elements_data_df.columns =  elements_data_df.c

,group_id,user_id,name,position,department,school,employee_id,net_id,is_current_staff,is_academic,...,doi,issn,eissn,types,type,scopus,web_of_science,crossref,pubmed,europe_pubmed_central
0,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,10.1080/01639374.2019.1699880,0163-9374,1544-4554,Journal Article,Journal article,2-s2.0-85076392535,NaN,10.1080/01639374.2019.1699880,NaN,NaN
1,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,10.1080/0361526X.2019.1583045,0361-526X,1541-1095,Journal Article,Journal article,2-s2.0-85065402087,NaN,10.1080/0361526x.2019.1583045,NaN,NaN
2,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,10.1080/08989621.2020.1779591,0898-9621,1545-5815,"Article, Journal",Journal article,2-s2.0-85087477966,WOS:000549691200001,10.1080/08989621.2020.1779591,32602379.0,MED:32602379
3,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,10.1074/jbc.W119.009587,NaN,1083-351X,Erratum,Journal article,2-s2.0-85068977968,NaN,10.1074/jbc.w119.009587,31253689.0,MED:31253689
4,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,10.1126/scitranslmed.abe4282,1946-6234,1946-6242,"Article, Journal",Journal article,2-s2.0-85097911743,WOS:000599969700007,10.1126/scitranslmed.abe4282,33257409.0,MED:33257409


In [4]:
## Check the length of the dataframe (i.e. how many rows)
print("Number of rows in the dataframe: ", len(elements_data_df.index))

## Check how many nans are in each column
elements_data_df.isnull().sum(axis = 0)

Number of rows in the dataframe:  69906


group_id                     0
user_id                      0
name                         0
position                     0
department                 265
school                      67
employee_id                  0
net_id                       0
is_current_staff             0
is_academic                  0
publication_id               0
publication-date             0
doi                       4395
issn                     10881
eissn                    10385
types                      837
type                         0
scopus                    6760
web_of_science           21445
crossref                  4239
pubmed                   10107
europe_pubmed_central     9933
dtype: int64

In [5]:
## Make a column based on year
## 1. Change publication_date column from object to a datetime format
elements_data_df['publication-date'] = pd.to_datetime(elements_data_df['publication-date'], format='%m/%d/%Y')
##elements_data_df['publication_date'] = pd.to_datetime(elements_data_df['publication_date'], format='%m/%d/%Y')

## 2. Extract year and create new column from publication_date
elements_data_df['year'] = elements_data_df['publication-date'].dt.year 
##elements_data_df['year'] = elements_data_df['publication_date'].dt.year 
elements_data_df.head()

,group_id,user_id,name,position,department,school,employee_id,net_id,is_current_staff,is_academic,...,issn,eissn,types,type,scopus,web_of_science,crossref,pubmed,europe_pubmed_central,year
0,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,0163-9374,1544-4554,Journal Article,Journal article,2-s2.0-85076392535,NaN,10.1080/01639374.2019.1699880,NaN,NaN,2020
1,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,0361-526X,1541-1095,Journal Article,Journal article,2-s2.0-85065402087,NaN,10.1080/0361526x.2019.1583045,NaN,NaN,2019
2,15,5,Violeta Ilik,"Department Head, Library",Galter Health Science Library;,Feinberg School of Medicine,1086765,viq454,0,1,...,0898-9621,1545-5815,"Article, Journal",Journal article,2-s2.0-85087477966,WOS:000549691200001,10.1080/08989621.2020.1779591,32602379.0,MED:32602379,2021
3,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,NaN,1083-351X,Erratum,Journal article,2-s2.0-85068977968,NaN,10.1074/jbc.w119.009587,31253689.0,MED:31253689,2019
4,15,8,Jon Lomasney,Associate Professor,Pathology;,Feinberg School of Medicine,1000184,jlomas,1,1,...,1946-6234,1946-6242,"Article, Journal",Journal article,2-s2.0-85097911743,WOS:000599969700007,10.1126/scitranslmed.abe4282,33257409.0,MED:33257409,2020


In [10]:
# Group by year and count unique IDs
unique_scopus_per_year = elements_data_df.groupby('year')['scopus'].nunique()
unique_wos_per_year = elements_data_df.groupby('year')['web_of_science'].nunique()

# Display the results
# print("Unique Scopus IDs per year:")
# print(unique_scopus_per_year)
# print("\nUnique Web of Science IDs per year:")
# print(unique_wos_per_year)

# # Combine the results into a new DataFrame
results_df = pd.DataFrame({
    'Unique Scopus IDs': unique_scopus_per_year,
    'Unique Web of Science IDs': unique_wos_per_year
}).reset_index()

# Display the results DataFrame
results_df.head()


,year,Unique Scopus IDs,Unique Web of Science IDs
0,2019,7942,6280
1,2020,8186,5709
2,2021,8439,5372
3,2022,7473,4702
4,2023,6711,4132


In [11]:
## Save dataframe to a CSV

with open(r"output/total_ids_year_df.csv", 'w', encoding='utf-8') as file:
    results_df.to_csv(file, line_terminator='\n', index=True)
    file.close()




In [20]:
# Print unique values from a specific column, e.g., 'types'
unique_types = elements_data_df['types'].unique()
# print("Unique types:")
# print(unique_types)

types = ['Journal Article' 
         ,'Article, Journal'
         ,'Review, Journal' 
         ,'Article, Data Paper, Journal'
         ,'Article, Early Access, Journal'
         ,'Article, Book Chapter, Book, Book in series'
         ,"Research Support, U.S. Gov't, P.H.S., Research Support, Non-U.S. Gov't, research-article, Research Support, U.S. Gov't, Non-P.H.S., Journal Article, Research Support, N.I.H., Extramural"
         ,'Chapter' 
         ,'Review' 
         ,'review'
         ,'Case Reports, case-report' 
         ,'Article'
         ,'research-article, Journal Article'
         ,'Biographical-Item, Biography, Journal'
         ,'Preprint'
         ,'Article, Book Chapter, Book in series'
         ,'Book'
         ,'Article, Proceedings Paper, Meeting, Journal'
         ,"Research Support, Non-U.S. Gov't, research-article, Journal Article, Research Support, N.I.H., Extramural"
         ,'Biographical-Item, Journal'
         ,'Article, Proceedings Paper, Journal'
         ,"Clinical Trial, Phase II, Research Support, Non-U.S. Gov't, research-article, Clinical Trial, Phase I, Journal Article"
         ,'Short Survey' 
         ,"Research Support, Non-U.S. Gov't, Journal Article"
         ,'meeting-report, Journal Article'
         ,'Review, Book Chapter, Book, Book in series'
         ,'research-article, Journal Article, Research Support, N.I.H., Extramural'
         ,'Article, Book Chapter, Book'
         ,'CPAPER'
         ,'GEN'
         ,'Multicenter Study, Journal Article'
         ,"Research Support, Non-U.S. Gov't, research-article, Multicenter Study, Journal Article, Research Support, N.I.H., Extramural"
         ,'Practice Guideline' 
         ,'review-article, Review, Journal Article'
         ,"Research Support, Non-U.S. Gov't, Randomized Controlled Trial, Journal Article"
         ,'Journal Article, case-report' 
         ,'Abstract'
         ,'Meta-Analysis, Comparative Study, Systematic Review, Journal Article'
         ,'Review, Early Access, Journal'
         ,'research-article, Preprint'
         ,"Research Support, Non-U.S. Gov't, research-article, Journal Article"
         ,'Journal'
         ,'Consensus Development Conference, research-article, Review, Journal Article'
         ,'Review, Journal Article' 
         ,'other, Practice Guideline, Journal Article'
         ,"Research Support, N.I.H., Intramural, Research Support, Non-U.S. Gov't, research-article, Journal Article, Research Support, N.I.H., Extramural"
         ,'JOUR' 
         ,'research-article, Multicenter Study, Journal Article'
         ,'Book Review, Review, Journal'
         ,'brief-report, Journal Article'
         ,"Research Support, Non-U.S. Gov't, Review, Journal Article"
         ,'Research Support, N.I.H., Intramural, research-article, Journal Article, Research Support, N.I.H., Extramural'
         ,'Practice Guideline, Journal Article'
         ,'Video-Audio Media, Journal Article' 
         ,'Book Review, Journal'
         ,'Review, Book Chapter, Book in series' 
         ,'Journal Article, Case Reports, case-report'
         ,"Clinical Trial, Research Support, U.S. Gov't, P.H.S., Research Support, Non-U.S. Gov't, research-article, Journal Article, Research Support, N.I.H., Extramural"
         ,'Systematic Review, Journal Article'
         ,'Journal Article, Observational Study'
         ,'research-article, Journal Article, Research Support, N.I.H., Extramural, Observational Study'
         ,"Research Support, Non-U.S. Gov't, Multicenter Study, Journal Article, Research Support, N.I.H., Extramural"
         ,'Journal Article, Research Support, N.I.H., Extramural'
         ,"Research Support, Non-U.S. Gov't, review-article, Review, Journal Article"
         ,"brief-report, Research Support, U.S. Gov't, P.H.S., Research Support, Non-U.S. Gov't, Journal Article, Research Support, N.I.H., Extramural"
         ,"Clinical Trial, Phase III, Research Support, Non-U.S. Gov't, research-article, Multicenter Study, Randomized Controlled Trial, Journal Article"
         ,'Review, Journal Article, Case Reports'
         ,'Reprint, Journal'
         ,"Research Support, Non-U.S. Gov't, Journal Article, Research Support, N.I.H., Extramural"
         ,"Research Support, Non-U.S. Gov't, research-article, Journal Article, Observational Study"
         ,'CONF'
         ,'Meta-Analysis, Systematic Review, Journal Article'
         ,'IM, Journal Article'
         ,"Research Support, Non-U.S. Gov't, Research Support, U.S. Gov't, Non-P.H.S., Journal Article, Research Support, N.I.H., Extramural"
         ,'research-article, Review, Journal Article'
         ,"Research Support, Non-U.S. Gov't, Clinical Study, Journal Article"
         ,'Journal Article, Research Support, N.I.H., Extramural, Observational Study'
         ,'research-article, Review, Journal Article, Research Support, N.I.H., Extramural'
         ,"Meta-Analysis, Research Support, Non-U.S. Gov't, research-article, Systematic Review, Review, Journal Article, Research Support, N.I.H., Extramural"
         ,'Comparative Study, Journal Article'
         ,'Article, Proceedings Paper, Meeting, Book in series' 
         ,'article']

print(types)

['Journal Article', 'Article, Journal', 'Review, Journal', 'Article, Data Paper, Journal', 'Article, Early Access, Journal', 'Article, Book Chapter, Book, Book in series', "Research Support, U.S. Gov't, P.H.S., Research Support, Non-U.S. Gov't, research-article, Research Support, U.S. Gov't, Non-P.H.S., Journal Article, Research Support, N.I.H., Extramural", 'Chapter', 'Review', 'review', 'Case Reports, case-report', 'Article', 'research-article, Journal Article', 'Biographical-Item, Biography, Journal', 'Preprint', 'Article, Book Chapter, Book in series', 'Book', 'Article, Proceedings Paper, Meeting, Journal', "Research Support, Non-U.S. Gov't, research-article, Journal Article, Research Support, N.I.H., Extramural", 'Biographical-Item, Journal', 'Article, Proceedings Paper, Journal', "Clinical Trial, Phase II, Research Support, Non-U.S. Gov't, research-article, Clinical Trial, Phase I, Journal Article", 'Short Survey', "Research Support, Non-U.S. Gov't, Journal Article", 'meeting-rep

In [24]:
# Function to check if a type is in the list or matches 'article' or 'review'
def type_matches(t, types):
    if isinstance(t, str):
        return t.lower() in ['article', 'review'] or t in types
    return False

# Apply the filter to the DataFrame
filtered_df = elements_data_df[elements_data_df['types'].apply(lambda x: type_matches(x, types))]

# Group by year and count unique IDs
unique_scopus_per_year = filtered_df.groupby('year')['scopus'].nunique()
unique_wos_per_year = filtered_df.groupby('year')['web_of_science'].nunique()

# Combine the results into a new DataFrame
results_df = pd.DataFrame({
    'Year': unique_scopus_per_year.index,
    'Unique Scopus IDs': unique_scopus_per_year.values,
    'Unique Web of Science IDs': unique_wos_per_year.values
})



# Display the results DataFrame
results_df.head()



,Year,Unique Scopus IDs,Unique Web of Science IDs
0,2019,6706,4504
1,2020,6772,4148
2,2021,7109,4130
3,2022,6312,3579
4,2023,5680,3231


In [25]:
## Save dataframe to a CSV

with open(r"output/total_art_ref_ids_year_df.csv", 'w', encoding='utf-8') as file:
    results_df.to_csv(file, line_terminator='\n', index=True)
    file.close()
